In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('daily_occupancy.csv')


df['fecha'] = pd.to_datetime(df['fecha'])
df = df.sort_values('fecha')
df.set_index('fecha', inplace=True)
serie = df['ocupacion']

In [ ]:
plt.figure(figsize=(16, 5))
plt.plot(serie, label='Ocupación diaria')
plt.title('Ocupación diaria a lo largo del tiempo')
plt.xlabel('Fecha')
plt.ylabel('Ocupación')
plt.legend()
plt.show()
